In [ ]:
%reload_ext eradiate.notebook
import matplotlib.pyplot as plt
import mitsuba as mi

import eradiate
from eradiate import KernelContext
from eradiate import unit_registry as ureg
from eradiate.kernel import SceneParameter, SearchSceneParameter
from eradiate.xarray.interp import dataarray_to_rgb

eradiate.set_mode("mono")

In [ ]:
# Define a callable that evaluates a spectrum

def spectrum_lambertian(ctx: KernelContext):
    if ctx.si.w == 440.0 * ureg.nm:
        return 0.1
    elif ctx.si.w == 550.0 * ureg.nm:
        return 0.5
    elif ctx.si.w == 660.0 * ureg.nm:
        return 0.9
    else:
        raise RuntimeError(f"Unsupported wavelength value {ctx.si.w = }")

In [ ]:
# Declare an Experiment with custom kdict and kpmap
exp = eradiate.experiments.AtmosphereExperiment(
    atmosphere={"type": "molecular"},
    surface={"type": "lambertian"},
    measures={
        "type": "perspective",
        "origin": [5, 5, 1],
        "target": [0, 0, 0.5],
        "film_resolution": (320, 240),
        "srf": {"type": "delta", "wavelengths": [440, 550, 660]},
    },
    kdict={
        # Declare materials separately, upstream
        "mat_laboule": {"type": "diffuse", "id": "mat_laboule"},  # Good practice: Match ID and dict key
        "mat_lecarre": {"type": "roughplastic", "id": "mat_lecarre"},
        # Then, declare geometric shapes
        "sphere": {
            "type": "sphere",
            "center": [0, 0, 0],
            "id": "laboule",
            "bsdf": {"type": "ref", "id": "mat_laboule"},  # Reference declared materials by ID
        },
        "square": {
            "type": "rectangle",
            "to_world": mi.ScalarTransform4f.translate([0, 0, 0.1])
            @ mi.ScalarTransform4f.scale(2.0),
            "bsdf": {"type": "ref", "id": "mat_lecarre"},
        },
    },
    kpmap={
        "mat_laboule.reflectance.value": SceneParameter(
            spectrum_lambertian,  # Callable to evaluate
            search=SearchSceneParameter(  # This information guides the scene tree traversal algorithm
                node_type=mi.BSDF,  # Search for a BSDF node in the graph ...
                node_id="mat_laboule",  # ... with ID 'mat_laboule' ...
                parameter_relpath="reflectance.value",  # ... that has a child parameter 'reflectance.value'
            ),
        ),
    },
)

In [ ]:
# View kdict
exp.kdict_full()

In [ ]:
# Inspect scene parameters to check against kpmap keys
exp.init()
exp.mi_scene.parameters

In [ ]:
# Render the scene
result = eradiate.run(exp, spp=256)

In [ ]:
img = dataarray_to_rgb(result.radiance, channels=[("w", 660), ("w", 550), ("w", 440)])
plt.axis("off")
plt.imshow(img)
plt.imsave("expert.png", img)
plt.show()
plt.close()